# 🧪 Script 09 — Benchmark BM25 sur corpus chunké (JSONL)

## 🎯 Pourquoi cette étape dans la progression ?
Après le **chunking XML (Script 08)**, l’objectif est de mesurer immédiatement l’impact du passage au **niveau “passage/chunk”** sur le retrieval.

- En BM25, indexer des documents trop longs augmente le bruit (beaucoup de tokens non discriminants).
- Le **chunking** crée des unités plus fines (articles / passages) : on s’attend à un **ranking plus propre** et des métriques (MRR/nDCG) qui montent.
- On conserve un protocole simple et comparable (mêmes requêtes et métriques), puis on teste des variantes :
  - requête brute
  - requête enrichie via `query_understanding.py`
  - (optionnel) **expansion minimale métier** (heuristique explicable)

---

SCRIPT 9 — BENCHMARK BM25 SUR CORPUS CHUNKÉ (JSONL)

But
---
Mesurer rapidement l'impact d'un preprocessing "passage-level" (Script 8) sur un retriever BM25 :
- Recall@k, MRR, nDCG@k
- audit qualitatif (verbatim top-k)

Pourquoi
--------
- Les chunks plus fins réduisent le bruit et améliorent le retrieval.
- BM25 reste très lexical : certaines requêtes "sémantiques" (Q1/Q3) peuvent échouer.
- On ajoute donc (optionnel) une "expansion minimale métier" pour augmenter la robustesse
  sans partir dans une refonte complète.

Modes évalués
-------------
1) BM25 — requête brute
2) BM25 — requête enrichie (query_understanding.py)
3) (Optionnel) BM25 — requête brute + expansion minimale métier
4) (Optionnel) BM25 — requête enrichie + expansion minimale métier

Notes d'implémentation
----------------------
- Chargement via corpus_loader_jsonl.py (load_documents + filter_documents_by_substring).
- Robustesse Spyder : parse_known_args() ignore les arguments inconnus.
- Déduplication optionnelle du top-k (évite d'afficher 10 variantes d'un même article).

## ⚙️ Setup notebook (imports locaux & dépendances)

Ce notebook exécute un script qui dépend de modules locaux (ex: `corpus_loader_jsonl.py`, `query_understanding.py`, etc.).  
En notebook, il faut s’assurer que la **racine du projet** est dans `sys.path`.

> 🔧 Astuce : place ce notebook dans le dossier projet, ou ajuste `PROJECT_ROOT` ci-dessous.


In [1]:
from pathlib import Path
import sys

# Ajuste PROJECT_ROOT si notre notebook est dans un sous-dossier (ex: notebooks/)
PROJECT_ROOT = Path.cwd()
# Exemple : si notre notebook est dans ./notebooks, décommente :
# PROJECT_ROOT = Path.cwd().parent

if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

print("PROJECT_ROOT =", PROJECT_ROOT)


PROJECT_ROOT = d:\-- Projet RAG Avocats --\codes_python\notebooks


In [2]:
# Dépendances externes (à installer une seule fois par environnement)
%pip -q install rank_bm25 pyyaml


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


## 🧠 Code du script 09 (inchangé)

Le code ci-dessous est repris du `.py` **tel quel** (à l’exception du fait qu’il est placé en cellule notebook).


In [3]:
from __future__ import annotations

import argparse
import re
from typing import Any, Dict, List, Optional, Sequence, Tuple

from rank_bm25 import BM25Okapi

from benchmark_queries import benchmark_queries
from metrics import recall_at_k, reciprocal_rank, ndcg_at_k
from query_understanding import load_juridical_dictionary, process_user_query

# Dépendance locale
from corpus_loader_jsonl import load_documents, filter_documents_by_substring


# =========================================================
# 1) Tokenisation BM25 (simple et stable)
# =========================================================

def tokenize(text: str) -> List[str]:
    """
    Tokenisation simple : minuscules + mots alphanumériques.

    Remarque :
    - BM25 étant lexical, garder une tokenisation simple est utile pour le debug.
    - On pourra ajouter stopwords plus tard si besoin.
    """
    return re.findall(r"\b\w+\b", (text or "").lower())


# =========================================================
# 2) Expansion minimale métier (optionnelle)
# =========================================================

def expand_query_minimal_legal(query: str) -> str:
    """
    Ajoute quelques termes "métier" très fréquents, de façon heuristique,
    pour réduire les échecs BM25 dus au mismatch lexical.

    Objectif :
    - rester petit / explicable / non invasif
    - aider surtout les requêtes type Q1/Q3 (rupture CDI sans préavis / contester licenciement)
    """
    q = (query or "").lower()
    expansions: List[str] = []

    # Contrat / CDI
    if "cdi" in q or "durée indéterminée" in q:
        expansions += [
            "contrat de travail à durée indéterminée",
            "rupture du contrat de travail",
        ]

    # Préavis / rupture sans préavis
    if "préavis" in q or ("sans" in q and "préavis" in q):
        expansions += [
            "faute grave",
            "faute lourde",
            "dispense de préavis",
        ]

    # Rupture (verbes fréquents)
    if "rompu" in q or "rompre" in q or "rupture" in q:
        expansions += [
            "licenciement",
            "démission",
            "prise d'acte",
        ]

    # Contestation / recours licenciement
    if "contester" in q or "contest" in q or "recours" in q:
        expansions += [
            "conseil de prud'hommes",
            "motif réel et sérieux",
            "indemnités",
        ]

    if "licenciement" in q:
        expansions += [
            "prud'hommes",
            "irrégulier",
            "nul",
        ]

    # Motif économique (pour compléter légèrement Q2)
    if "motif économique" in q or ("licenciement" in q and "économique" in q):
        expansions += [
            "difficultés économiques",
            "mutations technologiques",
            "sauvegarde de la compétitivité",
        ]

    # Normalisation : unicité + éviter d'ajouter trop de bruit
    cleaned: List[str] = []
    seen = set()
    for e in expansions:
        e = e.strip()
        if not e:
            continue
        if e in seen:
            continue
        seen.add(e)
        cleaned.append(e)

    if not cleaned:
        return query

    return (query.rstrip() + " " + " ".join(cleaned)).strip()


# =========================================================
# 3) Déduplication optionnelle du top-k
# =========================================================

def get_dedupe_value(doc: Dict[str, Any], dedupe_key: str) -> str:
    """
    Extrait une valeur stable pour dédupliquer un ranking.

    Exemples :
    - "meta.num" -> doc["meta"]["num"]
    - "meta.id"  -> doc["meta"]["id"]
    - "doc_id"   -> doc["doc_id"]
    """
    key = (dedupe_key or "").strip()
    if not key:
        return ""

    if key == "doc_id":
        return str(doc.get("doc_id") or "")

    if key.startswith("meta."):
        meta = doc.get("meta") or {}
        subkey = key.split(".", 1)[1]
        return str(meta.get(subkey) or "")

    return str(doc.get(key) or "")


def dedupe_ranked_results(
    ranked: List[Tuple[Dict[str, Any], float]],
    dedupe_key: str
) -> List[Tuple[Dict[str, Any], float]]:
    """
    Déduplique une liste triée (doc, score) en conservant le premier doc
    pour chaque valeur de dedupe_key.
    """
    key = (dedupe_key or "").strip()
    if not key:
        return ranked

    seen = set()
    out: List[Tuple[Dict[str, Any], float]] = []

    for doc, score in ranked:
        v = get_dedupe_value(doc, key)
        if not v:
            out.append((doc, score))
            continue
        if v in seen:
            continue
        seen.add(v)
        out.append((doc, score))

    return out


# =========================================================
# 4) BM25 search + évaluation
# =========================================================

def bm25_search(
    query: str,
    documents: Sequence[Dict[str, Any]],
    bm25: BM25Okapi,
    top_k: int,
    dedupe_key: str = ""
) -> List[Tuple[Dict[str, Any], float]]:
    """
    Retourne le top_k (doc, score) BM25.
    """
    scores = bm25.get_scores(tokenize(query))
    ranked = sorted(zip(documents, scores), key=lambda x: x[1], reverse=True)

    if dedupe_key:
        ranked = dedupe_ranked_results(ranked, dedupe_key)

    return ranked[:top_k]


def is_relevant_by_keywords(text: str, relevant_keywords: Sequence[str]) -> bool:
    """
    Oracle simple : pertinent si au moins un mot-clé du benchmark est présent.
    """
    t = (text or "").lower()
    return any((kw or "").lower() in t for kw in relevant_keywords)


def evaluate_bm25(
    documents: Sequence[Dict[str, Any]],
    bm25: BM25Okapi,
    k: int,
    use_query_understanding: bool,
    dictionary: Optional[Dict[str, Any]],
    use_min_expansion: bool,
    dedupe_key: str,
) -> Dict[str, float]:
    """
    Calcule Recall@k, MRR, nDCG@k sur benchmark_queries.
    """
    recall_scores: List[float] = []
    mrr_scores: List[float] = []
    ndcg_scores: List[float] = []

    for q in benchmark_queries:
        query_text = q["question"]

        if use_query_understanding:
            if dictionary is None:
                raise ValueError("dictionary requis si use_query_understanding=True")
            enriched = process_user_query(query_text, dictionary)
            query_text = enriched.get("enriched_query", query_text)

        if use_min_expansion:
            query_text = expand_query_minimal_legal(query_text)

        results = bm25_search(
            query=query_text,
            documents=documents,
            bm25=bm25,
            top_k=k,
            dedupe_key=dedupe_key,
        )

        recall_scores.append(recall_at_k(results, q["relevant_keywords"], k))
        mrr_scores.append(reciprocal_rank(results, q["relevant_keywords"]))
        ndcg_scores.append(ndcg_at_k(results, q["relevant_keywords"], k))

    n = max(1, len(benchmark_queries))
    return {
        f"Recall@{k}": sum(recall_scores) / n,
        "MRR": sum(mrr_scores) / n,
        f"nDCG@{k}": sum(ndcg_scores) / n,
    }


def display_results_per_query(
    label: str,
    documents: Sequence[Dict[str, Any]],
    bm25: BM25Okapi,
    k: int,
    use_query_understanding: bool,
    dictionary: Optional[Dict[str, Any]],
    use_min_expansion: bool,
    dedupe_key: str,
) -> None:
    """
    Affiche le top-k par requête (audit qualitatif).
    """
    print(f"\n=== VERBATIM — {label} ===")

    for q in benchmark_queries:
        base_query = q["question"]
        final_query = base_query

        if use_query_understanding:
            enriched = process_user_query(base_query, dictionary or {})
            final_query = enriched.get("enriched_query", base_query)

        if use_min_expansion:
            final_query = expand_query_minimal_legal(final_query)

        print("\n" + "=" * 110)
        print(f"ID: {q.get('id')} | Question: {base_query}")
        if final_query != base_query:
            print(f"Requête utilisée: {final_query}")
        print("=" * 110)

        results = bm25_search(
            query=final_query,
            documents=documents,
            bm25=bm25,
            top_k=k,
            dedupe_key=dedupe_key,
        )

        for rank, (doc, score) in enumerate(results, start=1):
            text = doc.get("text", "")
            relevant = is_relevant_by_keywords(text, q["relevant_keywords"])
            tag = "PERTINENT" if relevant else "NON PERTINENT"

            meta = doc.get("meta") or {}
            doc_type = doc.get("doc_type")
            chunk_index = doc.get("chunk_index")
            meta_id = meta.get("id")
            meta_num = meta.get("num")

            print(f"\n--- Rang {rank} | Score BM25: {score:.3f} | {tag}")
            print(f"doc_type={doc_type} | chunk_index={chunk_index} | meta.id={meta_id} | meta.num={meta_num}")
            print(f"doc_id: {doc.get('doc_id')}")
            print(text[:450].replace("\n", " ").strip())


# =========================================================
# 5) CLI / main
# =========================================================

def parse_args(argv: Optional[List[str]] = None) -> argparse.Namespace:
    """
    Parse des arguments CLI.

    Robustesse Spyder/IPython :
    - parse_known_args() ignore les arguments inconnus (évite SystemExit).
    """
    parser = argparse.ArgumentParser(add_help=True)

    parser.add_argument(
        "--corpus-jsonl",
        default=r"D:\-- Projet RAG Avocats --\data_main\result_tests\corpus_chunks.jsonl",
        help="Chemin du JSONL (sortie Script 8)."
    )
    parser.add_argument(
        "--dictionary-yml",
        default="juridical_dictionary.yml",
        help="Chemin du dictionnaire métier (YAML)."
    )
    parser.add_argument(
        "--k",
        type=int,
        default=10,
        help="Top-k pour Recall@k et nDCG@k."
    )
    parser.add_argument(
        "--doc-types",
        default="article",
        help='Doc types autorisés, séparés par virgule. Ex: "article,section_ta".'
    )
    parser.add_argument(
        "--title-contains",
        default="",
        help='Filtre métier: sous-chaîne à trouver dans meta.titre (ou doc_id fallback). Vide => pas de filtre.'
    )
    parser.add_argument(
        "--limit",
        type=int,
        default=None,
        help="Limiter le nombre de chunks chargés (debug)."
    )
    parser.add_argument(
        "--show-verbatim",
        action="store_true",
        help="Afficher le top-k par question."
    )
    parser.add_argument(
        "--dedupe-key",
        default="",
        help='Déduplique le ranking dans le top-k. Ex: "meta.num" ou "meta.id". Vide => pas de déduplication.'
    )
    parser.add_argument(
        "--min-expansion",
        action="store_true",
        help="Active l'expansion minimale métier (BM25 + expansion)."
    )

    args, _unknown = parser.parse_known_args(argv)
    return args


def parse_csv_list(value: str) -> List[str]:
    """
    Parse une liste CSV simple (séparée par virgule) en nettoyant les espaces.
    """
    if not value:
        return []
    return [v.strip() for v in value.split(",") if v.strip()]


def load_jsonl_documents_robust(jsonl_path: str, limit: Optional[int]) -> List[Dict[str, Any]]:
    """
    Chargement via corpus_loader_jsonl.load_documents, avec robustesse sur signature.

    On essaie plusieurs signatures possibles pour éviter de casser le script si le loader évolue.
    """
    # Essai 1 : loader unifié type load_documents(source="jsonl", path=..., ...)
    try:
        return load_documents(source="jsonl", path=jsonl_path, min_text_len=50, limit=limit)
    except TypeError:
        pass

    # Essai 2 : load_documents(jsonl_path, min_text_len=..., limit_lines=...)
    try:
        return load_documents(jsonl_path, min_text_len=50, limit_lines=limit)
    except TypeError:
        pass

    # Essai 3 : load_documents(jsonl_path) (mode minimal)
    return load_documents(jsonl_path)


def main() -> None:
    args = parse_args()
    
    # Overrides pratiques pour run dans Spyder (sans arguments)
    args.show_verbatim = True
    args.dedupe_key = "meta.num"   # ou "meta.id" selon ce qu'on veut
    args.min_expansion = True


    allowed_doc_types = [d.lower() for d in parse_csv_list(args.doc_types)]
    title_contains = (args.title_contains or "").strip() or None
    dedupe_key = (args.dedupe_key or "").strip()
    use_min_expansion = bool(args.min_expansion)

    print("\n=== Chargement corpus chunké (JSONL) ===")
    documents = load_jsonl_documents_robust(args.corpus_jsonl, limit=args.limit)
    print(f"Chunks chargés : {len(documents)}")

    # Filtre doc_types (simple)
    if allowed_doc_types:
        documents = [d for d in documents if (d.get("doc_type") or "").lower() in allowed_doc_types]

    # Filtre métier optionnel (titre / doc_id fallback)
    print("\n=== Filtrage (optionnel) ===")
    if title_contains:
        documents = filter_documents_by_substring(documents, title_contains, search_in=("meta.titre", "doc_id"))
        print(f"Filtre titre_contains : '{title_contains}'")

    print(f"Chunks après filtre : {len(documents)}")
    if allowed_doc_types:
        print(f"Filtre doc_types : {allowed_doc_types}")

    # BM25
    print("\n=== Construction index BM25 ===")
    tokenized_corpus = [tokenize(d.get("text", "")) for d in documents]
    bm25 = BM25Okapi(tokenized_corpus)
    print("Index BM25 construit.")

    # Dictionnaire métier pour query understanding
    dictionary = load_juridical_dictionary(args.dictionary_yml)

    # 1) Brute
    print("\n=== Benchmark BM25 sur chunks — requête brute ===")
    res_raw = evaluate_bm25(
        documents=documents,
        bm25=bm25,
        k=args.k,
        use_query_understanding=False,
        dictionary=None,
        use_min_expansion=False,
        dedupe_key=dedupe_key,
    )
    print(f"Recall@{args.k} : {res_raw[f'Recall@{args.k}']:.3f}")
    print(f"MRR : {res_raw['MRR']:.3f}")
    print(f"nDCG@{args.k} : {res_raw[f'nDCG@{args.k}']:.3f}")

    # 2) Query understanding
    print("\n=== Benchmark BM25 sur chunks — requête enrichie (query understanding) ===")
    res_enriched = evaluate_bm25(
        documents=documents,
        bm25=bm25,
        k=args.k,
        use_query_understanding=True,
        dictionary=dictionary,
        use_min_expansion=False,
        dedupe_key=dedupe_key,
    )
    print(f"Recall@{args.k} : {res_enriched[f'Recall@{args.k}']:.3f}")
    print(f"MRR : {res_enriched['MRR']:.3f}")
    print(f"nDCG@{args.k} : {res_enriched[f'nDCG@{args.k}']:.3f}")

    # 3) Expansion minimale (optionnelle)
    if use_min_expansion:
        print("\n=== Benchmark BM25 sur chunks — requête brute + expansion minimale métier ===")
        res_exp = evaluate_bm25(
            documents=documents,
            bm25=bm25,
            k=args.k,
            use_query_understanding=False,
            dictionary=None,
            use_min_expansion=True,
            dedupe_key=dedupe_key,
        )
        print(f"Recall@{args.k} : {res_exp[f'Recall@{args.k}']:.3f}")
        print(f"MRR : {res_exp['MRR']:.3f}")
        print(f"nDCG@{args.k} : {res_exp[f'nDCG@{args.k}']:.3f}")

        print("\n=== Benchmark BM25 sur chunks — requête enrichie + expansion minimale métier ===")
        res_enriched_exp = evaluate_bm25(
            documents=documents,
            bm25=bm25,
            k=args.k,
            use_query_understanding=True,
            dictionary=dictionary,
            use_min_expansion=True,
            dedupe_key=dedupe_key,
        )
        print(f"Recall@{args.k} : {res_enriched_exp[f'Recall@{args.k}']:.3f}")
        print(f"MRR : {res_enriched_exp['MRR']:.3f}")
        print(f"nDCG@{args.k} : {res_enriched_exp[f'nDCG@{args.k}']:.3f}")

    # Verbatim
    if args.show_verbatim:
        display_results_per_query(
            label="Requête brute",
            documents=documents,
            bm25=bm25,
            k=args.k,
            use_query_understanding=False,
            dictionary=None,
            use_min_expansion=False,
            dedupe_key=dedupe_key,
        )

        display_results_per_query(
            label="Requête enrichie",
            documents=documents,
            bm25=bm25,
            k=args.k,
            use_query_understanding=True,
            dictionary=dictionary,
            use_min_expansion=False,
            dedupe_key=dedupe_key,
        )

        if use_min_expansion:
            display_results_per_query(
                label="Requête brute + expansion minimale",
                documents=documents,
                bm25=bm25,
                k=args.k,
                use_query_understanding=False,
                dictionary=None,
                use_min_expansion=True,
                dedupe_key=dedupe_key,
            )

            display_results_per_query(
                label="Requête enrichie + expansion minimale",
                documents=documents,
                bm25=bm25,
                k=args.k,
                use_query_understanding=True,
                dictionary=dictionary,
                use_min_expansion=True,
                dedupe_key=dedupe_key,
            )


if __name__ == "__main__":
    main()


=== Chargement corpus chunké (JSONL) ===
Chunks chargés : 13180

=== Filtrage (optionnel) ===
Chunks après filtre : 11323
Filtre doc_types : ['article']

=== Construction index BM25 ===
Index BM25 construit.

=== Benchmark BM25 sur chunks — requête brute ===
Recall@10 : 0.667
MRR : 0.444
nDCG@10 : 0.500

=== Benchmark BM25 sur chunks — requête enrichie (query understanding) ===
Recall@10 : 0.667
MRR : 0.444
nDCG@10 : 0.500

=== Benchmark BM25 sur chunks — requête brute + expansion minimale métier ===
Recall@10 : 0.667
MRR : 0.667
nDCG@10 : 0.667

=== Benchmark BM25 sur chunks — requête enrichie + expansion minimale métier ===
Recall@10 : 0.667
MRR : 0.667
nDCG@10 : 0.667

=== VERBATIM — Requête brute ===

ID: Q1 | Question: Dans quels cas un CDI peut-il être rompu sans préavis ?

--- Rang 1 | Score BM25: 20.945 | NON PERTINENT
doc_type=article | chunk_index=0 | meta.id=None | meta.num=L3142-17
doc_id: D:\-- Projet RAG Avocats --\data_main\data\20250731-220719\legi\global\code_et_TNC_e

## ▶️ Exécution (optionnelle)

Par défaut, le script utilise un chemin Windows en argument `--corpus-jsonl`.  
En notebook, on peut surcharger les arguments en définissant `sys.argv` avant d’appeler `main()`.

⚠️ Pense à mettre le chemin réel vers notre `corpus_chunks.jsonl` (sortie Script 08).


In [4]:
import sys

# Exemple : surcharge des arguments CLI (à adapter)
# sys.argv = [
#     "",
#     "--corpus-jsonl", r"D:\-- Projet RAG Avocats --\data_main\result_tests\corpus_chunks.jsonl",
#     "--k", "10",
#     "--doc-types", "article",
#     "--show-verbatim",
#     "--dedupe-key", "meta.num",
#     "--min-expansion",
# ]

# Lance le main
# main()


## Analyse des résultats — Script 9 (BM25 sur corpus chunké)

### Protocole
- **Corpus** : `corpus_chunks.jsonl` (sortie Script 8 : chunking *passage-level* + nettoyage d’en-têtes).
- **Filtrage métier** : **désactivé** (important : un filtre type `"code du travail"` peut artificiellement faire chuter les scores en excluant trop de chunks).
- **Doc types indexés** : `article` uniquement (pour limiter le bruit “plan/structure”).
- **Benchmark** : inchangé (`benchmark_queries.py`, 3 requêtes Q1–Q3).
- **Métriques** : Recall@10, MRR, nDCG@10 (`metrics.py`).
- **Options de debug** :
  - `show_verbatim=True` pour audit qualitatif.
  - `dedupe_key` activable (ex: `meta.num`) pour éviter d’afficher/compter des quasi-doublons.

### Résultats
| Configuration                                                  | Recall@10 | MRR   | nDCG@10 |
|---|---:|---:|---:|
| BM25 chunks — requête brute                                    | 0.667 | 0.444 | 0.500 |
| BM25 chunks — requête enrichie (query understanding)           | 0.667 | 0.444 | 0.500 |
| BM25 chunks — requête brute + **expansion minimale métier**    | 0.667 | 0.667 | 0.667 |
| BM25 chunks — enrichie + **expansion minimale métier**         | 0.667 | 0.667 | 0.667 |

### Lecture / Interprétation (quantitatif)
- **Recall@10 stable (0.667)** : on récupère toujours **2 requêtes sur 3** dans le top-10.
- **Gain fort sur MRR et nDCG** avec l’**expansion minimale** :
  - **MRR 0.444 → 0.667** : quand on trouve, on trouve **plus haut** (souvent rang 1).
  - **nDCG@10 0.500 → 0.667** : le top-10 est **mieux ordonné**, avec plus de pertinents en haut.

### Lecture / Interprétation (qualitatif — verbatim)
- **Q2 (licenciement pour motif économique)** : bon comportement (articles très “lexicalisés”, BM25 fonctionne bien).
- **Q1 (CDI rompu sans préavis)** : échec persistant même après expansion :
  - L’expansion ajoute des bons concepts (faute grave, faute lourde, dispense de préavis…),
  - mais BM25 remonte surtout des passages “structure du code / plan” qui matchent fortement (`rupture`, `contrat`, `licenciement`),
  - sans nécessairement tomber sur l’article normatif attendu sur le préavis.
- **Q3 (contester un licenciement)** : l’expansion aide le ranking global, mais l’audit est indispensable pour vérifier que les “PERTINENT” le sont vraiment (limites d’un oracle basé sur mots-clés).

### Points clés à retenir (à valoriser en entretien)
1. **Le chunking passage-level** rend le retrieval plus exploitable : passages plus courts, moins de bruit, et surtout **meilleur ranking**.
2. Une **expansion métier minimale** (même simple) peut améliorer **fortement** MRR/nDCG sans changer de modèle.
3. Les échecs restants (Q1) illustrent une limite attendue de BM25 : **décalage entre langage utilisateur et formulation juridique** + “bruit structurel”.
   → Justification naturelle d’un **dense retrieval** ou **hybride BM25 + dense rerank**, et/ou d’un filtrage plus fin des passages “plan de code”.

### Pistes d’amélioration rapides (sans sur-optimiser)
- **Réduire le bruit “plan/structure”** : détecter ces passages (patterns type “Partie / Livre / Titre / Chapitre / Section” sans contenu normatif) et les exclure ou les downweight.
- **Améliorer l’évaluation** : passer progressivement d’un oracle “mots-clés” à une cible plus robuste (`meta.num`, `meta.id`, ou mapping question → article(s) attendus).
- **Étape suivante logique** : tester un **dense retrieval sur chunks** (ou rerank dense) pour résoudre les requêtes “sémantiques” type Q1.